<center>
    <h1> INF335 - Tecnologías de Búsqueda Web   </h1>
    <h2> Tarea 1 </h2>
    <h3> Universidad Técnica Federico Santa Maria </h3>
    
</center>

_10 de Abril de 2017_
<p>Sebastián Aranda 201104560-2 <br> Felipe Santander 201104</p>

<h1>Introducción</h1>
<p>Se presenta un dataset con alrededor de 560.000 críticas de platos de comida y restaurantes provenientes de Amazon, para el cual se desea realizar procesamiento de texto a las reseñas utilizando la herramienta de python NLTK. Se propone un código compuesto por 3 scripts:</p>
<ul>
    <li><b>t1.1_tbw.py</b> Construcción del corpus</li>
    <li><b>t1.2_tbw.py</b> Filtración de tokens con baja frecuencia, Obtención de collocations, Postagging y Reconocimiento de entidades</li>
    <li><b>t1.3_tbw.py</b> Análisis de Sentimientos</li>
</ul>
<p>Es importante señalar que debido al gran tamaño del dataset provisto se decidió trabajar con un subconjunto de la totalidad de reseñas. Para lo anterior se separó el corpus en 56 subconjuntos con 10.000 reseñas cada uno. Utilizando la libreria Pickle se fueron guardando en el disco las particiones del corpus para su posterior uso.</p>

<h1>Procesamiento de Texto</h1>
<h2>Construcción del Corpus</h2>
<p>Para realizar el procesamiento de texto es necesario construir un Corpus. Para esto se realizaron los pasos descritos a continuación:</p>
<ol>
	<li><b>Eiminación de etiquetas HTML</b><p style="margin:0">Utilizando la librearia BeautifulSoup se eliminan los tags HTML presentes en algunas reseñas</p></li>
    <li><b>Transformación a minúsculas</b><p style="margin:0">Se transforma cada reseña a minúsculas para coincidir tokens</p></li>
    <li><b>Eliminación de signos</b><p style="margin:0">Se establece un listado de simbolos no permitidos y estos son eliminados de cada reseña</p></li>
    <li><b>Guardado de estado de corpus</b><p style="margin:0">Cada 10.000 reseñas procesadas se genera un archivo .pkl que guarda los documentos (reseñas), tokens filtrados y frecuencias del corpus generado con la cantidad de reseñas mencionada</p></li>
</ol>
<p>El proceso anterior se realizó para la generación de un solo Corpus, es decir, se procesaron solo 10.000 reseñas. Esta ejecución tardó <b>49,363 segundos</b>.</p>
<h1>Procesamiento Corpus</h1>
<p>Debido al elevado tiempo de ejecución de los distintos tipos de procesamiento del corpus se decidió trabajar con el siguiente conjunto de datos:</p>
<ul>
<li>Corpus de <b>10.000</b> Reseñas (primeras 10.000)</li>
<li>Totalidad de tokens del corpus de 10.000 reseñas: <b>384.004</b></li>
<li>Cantidad de documentos para reconocimiento de estructuras: <b>100</b></li>
<li>Tokens por documento promedio: <b>75</b></li>
<li>Tokens por documento máximo: <b>1.464</b></li>
<li>Tokens por documento mínimo: <b>10</b></li>
</ul>
<p>La lectura del corpus demoró <b>18,052 segundos</b>.</p>

<h2>Filtración de tokens poco frecuentes</h2>
<p>Se realizó un procedimiento similar a la eliminiación de stopwords, pero esta vez considerando las palabras que aparecen en el diccionario de frecuencia filtrado por los tokens que aparecen 3 o más veces.</p>
<p>El filtrado anterior tardó <b>454,981 segundos</b>.</p>

<h2>Obtención de Collocations</h2>
<p>Se procede leyendo el primer Corpus generado con 10.000 reseñas, luego con el listado de tokens se realiza una identificación de bigramas, seleccionando los 30 más frecuentes. Para lo anterior se utilizaron las funciones provistas por nltk para obtención de bigramas.</p>
<p>La identificación de bigramas tardó <b>4,943 segundos</b>.</p>

<h2>Postagging</h2>
<p>Se realizo postagging de 100 documentos. Analizando los resultados obtenidos se aprecia que el algoritmo etiqueta los tokens de manera correcta el x% de los tokens, considerando un conjunto de y tokens.</p>
<p>El procesamiento anterior demoró en promedio <b>6,507 segundos</b> por documento, si se hubiese intentado procesar el corpus seleccionado (10.000 documentos) el tiempo de ejecución hubiese sido de <b>18 horas</b> aprox.</p>

<h2>Reconocimiento de Entidades</h2>
<p>Al igual que para Postagging, se realizó el reconocimiento de entidades para 100 reseñas. Sin embargo se logra notar que es un poco más dificil reconocer entidades, demorando <b>7,872 segundos</b> por documento en promedio, por lo tanto se hubiese demorado alrededor de 20 horas en procesar todos los docuemntos del corpus</p>
<p>En cuanto a la eficacia del algoritmo se observa un x% de exactitud</p>
<p>En un principio se intentó realizar el reconocimiento con los documentos en minúsculas y el algoritmo no produjo ningún efecto.</p>

<h2>Análisis de Sentimientos</h2>
<p></p>

<h1>Conclusiones</h1>
<p>Se concluye...</p>

<h1>Anexo</h1>

<p><b>t1.1_tbw.py</b></p>

In [ ]:
import csv
import nltk
import pickle
import gc
import sys
import time
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from collections import defaultdict

stop = stopwords.words('english')
puncts = ".,:;?!()[]{}~+-\"\'#$%&"
#puncts = ".,?!()[]~-\"\'&1234567890"

file_path = './amazon-fine-foods/Reviews.csv'

timer = time.time()*1000
mcorpus = dict()
mcorpus['documents']= list()  #Corpus para guardar resenias
mcorpus['tokens'] = list() #Corpus para guardar tokens
mcorpus['token_freq'] = defaultdict(int) #Diccionario para contar repeticiones de tokens
i=-1
j=0
n = 10000 #Documentos procesados
with open(file_path, 'rb') as csvfile:
	spamreader = csv.reader(csvfile, delimiter=',')
	for row in spamreader: 
		i += 1
		if i==0:
			continue
		elif i>n:
			break

		#Eliminando etiquetas
		text = BeautifulSoup(row[9], 'html.parser').getText()

		#Guardando documento
		mcorpus['documents'].append(text.split())
		
		#Pasando a minusculas
		text = text.lower()

		#Eliminando signos y puntuaciones
		for sym in puncts:
			text = text.replace(sym, ' ')

		#Eliminando Stopwords
		filtered = [token for token in text.split() if token not in stop]
			
		for token in filtered:
			mcorpus['tokens'].append(token)
			mcorpus['token_freq'][token] += 1

		sys.stdout.write("\rDocumentos Procesados: "+str(i)+"/"+str(n))
		sys.stdout.flush()

		# Guardando avance en pickle cada 10000 registros
		if i%10000 == 0:
			j += 1
			pickle_file_path = './corpus/corpus_'+str(j)+'.pkl'
			output = open(pickle_file_path, 'wb')
			pickle.dump(mcorpus, output)
			output.close()
			
			mcorpus['documents']= list()  #Corpus para guardar resenias
			mcorpus['tokens'] = list() #Corpus para guardar tokens
			mcorpus['token_freq'] = defaultdict(int) #Diccionario para contar repeticiones de tokens
			gc.collect()
	
	timer = time.time()*1000-timer
	print('\nTiempo de procesamiento: ' + str(timer) + ' ms')

<p><b>t1.2_tbw.py</b></p>

In [ ]:
import nltk
import pickle
import gc
import time
import sys
from nltk.collocations import *
from collections import defaultdict
from nltk.tag import StanfordPOSTagger
from nltk.tag import StanfordNERTagger


#Funcion que elimina token con repeticiones menores a x
def dict_tokens_modified(data, x):
    new_dict_tokens = {k: v for k, v in data.iteritems() if v >= x}
    return new_dict_tokens

log = {'corpus_read_time': 0, 'filtering_infrequent_tokens_time': 0, 'top30bigrams_time': 0, 'postagger_time': 0, 'nertagger_time': 0}

total_timer = time.time()*1000

#Leyendo corpus desde archivo pickle (10.000 registros) creado previamente
print("Leyendo Corpus...")
corpus_timer = time.time()*1000
bigcorpus = dict()
bigcorpus['documents']= list()  #Corpus para guardar resenias
bigcorpus['tokens'] = list() #Corpus para guardar tokens
bigcorpus['token_freq'] = defaultdict(int) #Diccionario para contar repeticiones de tokens
tokens_per_document = list()
for i in range(1):
	i += 1
	pkl_file = open('./corpus/corpus_'+str(i)+'.pkl', 'rb')
	mcorpus = pickle.load(pkl_file) 
	pkl_file.close()

	bigcorpus['documents'] += mcorpus['documents'][:100]
	for doc in mcorpus['documents']:
		tokens_per_document.append(len(doc))
	bigcorpus['tokens'] += mcorpus['tokens']
	bigcorpus['token_freq'] = {k: bigcorpus['token_freq'].get(k,0)+mcorpus['token_freq'].get(k,0) for k in set(bigcorpus['token_freq']) | set(mcorpus['token_freq'])}

corpus_timer = time.time()*1000 - corpus_timer
log['corpus_read_time'] = corpus_timer

print("Largo Conjunto de Tokens del Corpus: "+str(len(bigcorpus['tokens'])))
print("Tokens por Documento Promedio: "+str(sum(tokens_per_document)/len(tokens_per_document)))
print("Tokens por Documento Maximo: "+str(max(tokens_per_document)))
print("Tokens por Documento Minimo: "+str(min(tokens_per_document)))

#Eliminando palabras con frecuencia menor a 3
print("Filtrando palabras con frecuencia menor a 3...")
filter_infreq_timer = time.time()*1000
filtered_dict = dict_tokens_modified(bigcorpus['token_freq'], 3)
validated_tokens = list(filtered_dict)
bigcorpus['tokens'] = [token for token in bigcorpus['tokens'] if token in validated_tokens]
bigcorpus['token_freq'] = filtered_dict

filter_infreq_timer = time.time()*1000-filter_infreq_timer
log['filtering_infrequent_tokens_time'] = filter_infreq_timer

#Determinando Top30 Collocations
print("Determinando Top30 Collocations...")
bigram_timer = time.time()*1000
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(bigcorpus['tokens'])
finder.apply_freq_filter(3)
bigrams = finder.nbest(bigram_measures.pmi, 30)

bigram_timer = time.time()*1000 - bigram_timer
log['top30bigrams_time'] = bigram_timer

print('\nTOP 30 Collocations Amazon Fine Foods Reviews\n')
i = 0
for bigram in bigrams:
	i += 1
	print str(i)+'. '+str(bigram)

print('\n')
#Ejecutando Postagger y Nertagger con n documentos
postagger = StanfordPOSTagger('./stanford-postagger/models/english-bidirectional-distsim.tagger', './stanford-postagger/stanford-postagger.jar', encoding='utf-8')
nertagger = StanfordNERTagger('./stanford-nertagger/classifiers/english.all.3class.distsim.crf.ser.gz', './stanford-nertagger/stanford-ner.jar', encoding='utf-8')
postagger_times = list()
nertagger_times = list()
i = 0
n = 100 #Cantidad de documentos a taggear
print("Ejecutando Postagger y Nertagger...")
for doc in bigcorpus['documents'][i:n]:
	i += 1
	pickle_file_path_postag = './postags/postagged_'+str(i)+'.pkl'
	postagger_time = time.time()*1000
	postagged = postagger.tag(doc)
	postagger_time = time.time()*1000 - postagger_time
	postagger_times.append(postagger_time)
	output = open(pickle_file_path_postag, 'wb')
	pickle.dump(postagged, output)
	output.close()

	pickle_file_path_nertag = './nertags/nertagged_'+str(i)+'.pkl'
	nertagger_time = time.time()*1000
	nertagged = nertagger.tag(doc)
	nertagger_time = time.time()*1000 - nertagger_time
	nertagger_times.append(nertagger_time)
	output = open(pickle_file_path_nertag, 'wb')
	pickle.dump(nertagged, output)

	postagged = None
	nertagged = None
	gc.collect()

	sys.stdout.write("\rDocumentos Procesados: "+str(i)+"/"+str(n))
	sys.stdout.flush()

log['postagger_time'] = sum(postagger_times)/len(postagger_times)
log['nertagger_time'] = sum(nertagger_times)/len(nertagger_times)

pickle_file_path = './log.pkl'
output = open(pickle_file_path, 'wb')
pickle.dump(log, output)
output.close()
print('\n')

print(log)
total_timer = time.time()*1000 - total_timer
print('Largo corpus documentos: '+str(len(bigcorpus['documents']))+ ' Largo corpus tokens:'+str(len(bigcorpus['tokens'])))
print("Tiempo de procesamiento Total: "+str(total_timer) + " ms")

<p><b>t1.3_tbw.py</b></p>

In [ ]:
import pickle
import gc
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
from collections import defaultdict
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize
from bs4 import BeautifulSoup
from nltk.corpus import stopwords

total_timer = time.time()*1000

#Leyendo corpus desde archivo pickle (10.000 registros) creado previamente
print("Leyendo Corpus...")
corpus_timer = time.time()*1000
bigcorpus = dict()
bigcorpus['documents'] = list()
for i in range(1):
	i += 1
	pkl_file = open('./corpus/corpus_'+str(i)+'.pkl', 'rb')
	mcorpus = pickle.load(pkl_file) 
	pkl_file.close()
	bigcorpus['documents'] += mcorpus['documents'][:100]

corpus_timer = time.time()*1000 - corpus_timer
print('\nTiempo de procesamiento: ' + str(corpus_timer) + ' ms')

#Entrena el sentiment_analyzer para ser usado en el corpus de resenias
print("Entrenando sentiment_analyzer...")
n_instances = 1000
subj_docs = [(sent, 'subj') for sent in subjectivity.sents(categories='subj')[:n_instances]]
obj_docs = [(sent, 'obj') for sent in subjectivity.sents(categories='obj')[:n_instances]]
train_subj_docs = subj_docs[:750]
test_subj_docs = subj_docs[750:1000]
train_obj_docs = obj_docs[:750]
test_obj_docs = obj_docs[750:1000]
sentiment_analyzer = SentimentAnalyzer()
training_docs = train_subj_docs+train_obj_docs
test_docs = test_subj_docs+test_obj_docs
training_set = sentiment_analyzer.apply_features(training_docs)
test_set = sentiment_analyzer.apply_features(test_docs)
trainer = NaiveBayesClassifier.train
classifier = sentiment_analyzer.train(trainer, training_set)
for key,value in sorted(sentiment_analyzer.evaluate(test_set).items()):
	print('{0}: {1}'.format(key, value))

#Se evalua la polaridad de las resenias cargadas en bigcorpus['documents']
print("Evaluando...")
pickle_file_path = './polarity/polarity.pkl'
output = open(pickle_file_path, 'wb')
stop = stopwords.words('english')
polarity = list()
sid = SentimentIntensityAnalyzer()
for sent in bigcorpus['documents']:
        lower = [token.lower() for token in sent]
        sentence_filtered = [token for token in lower if token not in stop]
        ss = sid.polarity_scores(" ".join(sentence_filtered))
        polarity.append(ss)
        #Descomentar para tener resultados por pantalla
        for k in sorted(ss):
                print('{0} : {1}'.format(k, ss[k]))
pickle.dump(polarity, output)
output.close()
gc.collect()
print("Terminado")